In [ ]:
# Before beginning this notebook you will need to add the following code to the query tool in pgAdmin: 
# ALTER TABLE counties ADD PRIMARY KEY (fips_date); ALTER TABLE hospitals ADD PRIMARY KEY (fips_date); ALTER TABLE vaccinations ADD PRIMARY KEY (fips_date);
# Additionally, you will need to enter your password into the connection_string variable in the config.py file. 
# The place holder text is YOURPASSWORDHERE

In [1]:
# Import dependencies
import sqlalchemy
from sqlalchemy import create_engine
import config
import pandas as pd
import psycopg2 as pg
import numpy as np

In [2]:
 # Create engine using the database file
engine = sqlalchemy.create_engine(config.connection_string)

In [3]:
# Create the connection
dbConnection = engine.connect();

In [4]:
# Join counties and hospitals tables into dataframe
merged_df = pd.read_sql("SELECT * FROM counties JOIN hospitals ON counties.fips_date = hospitals.fips_date", dbConnection);

In [5]:
# Create dataframe of vaccinations table
vaccinations_df = pd.read_sql("SELECT * FROM vaccinations", dbConnection);

In [6]:
# Create a list of the dataframes
dfs = [merged_df, vaccinations_df]

In [7]:
merged_df

,index,cases_to_date,deaths_to_date,fips_date,index,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,...,total_ped_patients_hospitalized_con_and_sus_covid_7_day_sum,total_ped_patients_hospitalized_con_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_con_and_sus_covid_7_day_sum,staffed_icu_adult_patients_con_covid_7_day_sum,fips_date
0,21,26881,314,10012020-12-25,21,1001,2020-12-25,581,581,462,...,0,0,462,42,42,42,42,27,27,10012020-12-25
1,24,34774,382,10012021-01-15,24,1001,2021-01-15,602,602,462,...,0,0,462,42,42,42,42,39,39,10012021-01-15
2,25,36617,397,10012021-01-22,25,1001,2021-01-22,602,602,462,...,0,0,462,42,42,42,42,41,40,10012021-01-22
3,31,44055,639,10012021-03-05,31,1001,2021-03-05,602,602,462,...,0,0,462,42,42,40,40,22,22,10012021-03-05
4,33,45263,672,10012021-03-19,33,1001,2021-03-19,602,602,462,...,0,0,462,42,42,33,33,9,9,10012021-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110337,208898,1738,7,780202021-06-18,159852,78020,2021-06-18,924,868,728,...,0,0,784,56,56,45,45,0,0,780202021-06-18
110338,208902,1772,7,780202021-07-16,159856,78020,2021-07-16,924,868,728,...,0,0,784,56,56,35,35,14,14,780202021-07-16
110339,208911,2242,21,780202021-09-17,159865,78020,2021-09-17,924,868,728,...,0,0,784,56,56,54,54,0,0,780202021-09-17
110340,208912,2258,21,780202021-09-24,159866,78020,2021-09-24,924,868,728,...,0,0,784,56,56,29,29,7,7,780202021-09-24


In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]

In [ ]:
column_numbers .remove(0) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]
column_numbers .remove(2) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
column_numbers = [x for x in range(merged_df.shape[1])]
column_numbers .remove(2) #removing column integer index 0
merged_df = merged_df.iloc[:, column_numbers] #return all columns except the 0th column
merged_df

In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:
# Create ratio columns for bed usage (ICU and inpatient)

In [ ]:
for col in vaccinations_df.columns:
    print(col)

In [ ]:
ml_ready_df = merged_df.merge(vaccinations_df, how='inner', on='fips_date')
ml_ready_df

In [ ]:
for col in ml_ready_df.columns:
    print(col)

In [ ]:
ml_ready_df = ml_ready_df.drop('index', 1)
ml_ready_df

In [ ]:
ml_ready_df["Percentage_inpatient_beds_used"] = ml_ready_df["inpatient_beds_used_7_day_sum"]/ml_ready_df["inpatient_beds_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["Percentage_ICU_beds_used"] = ml_ready_df["icu_beds_used_7_day_sum"]/ml_ready_df["total_icu_beds_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["total_patients_hospitalized_con_and_sus_covid_7_day_sum"] = ml_ready_df["total_adult_patients_hospitalized_con_and_sus_covid_7_day_sum"]+ml_ready_df["total_ped_patients_hospitalized_con_and_sus_covid_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df["total_patients_hospitalized_con_covid_7_day_sum"] = ml_ready_df["total_adult_patients_hospitalized_con_covid_7_day_sum"]+ml_ready_df["total_ped_patients_hospitalized_con_covid_7_day_sum"]
ml_ready_df

In [ ]:
ml_ready_df.replace([np.inf, -np.inf], np.nan)

In [ ]:
ml_ready_df.dropna()

In [ ]:
ml_ready_df.dtypes

In [ ]:
ml_ready_df = ml_ready_df.astype({'fips_code': 'int64'})

In [ ]:
ml_ready_df.dtypes

In [ ]:
ml_ready_df.sort_values(by=['fips_date'], inplace=True)

Data Exploration Charts

In [ ]:
multnomah_df = ml_ready_df[ml_ready_df["fips_code"] == 41051]
multnomah_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="Percentage_inpatient_beds_used", figsize=(20, 20))
plt.ylim(top=1.0)
plt.ylim(bottom=0.4)
plt.savefig('Percentage of Inpatient Beds Used Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="cases_to_date", figsize=(20, 20))
plt.savefig('Cases to Date Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="deaths_to_date", figsize=(20, 20))
plt.savefig('Deaths to Date Multnomah County')
plt.show()

In [ ]:
multnomah_df.plot.bar(x="collection_week", y="Series_Complete_Pop_Pct", figsize=(20, 20))
plt.savefig('Percent of Population Fully Vaccinated Multnomah County')
plt.show()